# Параллельная разработка с библиотекой `asyncio`

## Основы

Здесь сначала введем новый термин - конкурентность (от англ. concurrency). Почти везде слово concurrency на русский переводится как параллелизм, однако это не совсем корректный перевод. Параллелизм - это когда одновременно работают несколько потоков операций. Допустим, у нас есть ПК с 4 ядрами ЦПУ. Тогда параллельно могут работать не более 4 потоков. При этом, если посмотреть на диспетчер задач, то там скорее всего будет числиться гораздо большее число одновременно выполняемых процессов. То есть, операционная система выделяет каждому из потоков этих процессов какой-то квант процессорного времени, чередуя потоки. Это и есть пример конкурентности. 

В отличие от `concurrent.futures` библиотека `asyncio` реализует многозадачность через конкурентность с помощью корутин и использования цикла обработки событий. Начнем наше знакомство с `asyncio` с изучения различий с явной работой с потоками. Для этого реализуем один и тот же функционал двумя способами: сначала через параллелизм, а потом - через конкурентность.

In [5]:
%%writefile spinner_thread.py

import threading
import time
import sys
import itertools

class Signal:
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|/-\\'):
        status = f'{char} {msg}'
        write(status)
        flush()
        write('\x08' * len(status))
        time.sleep(.1)
        if not signal.go:
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    time.sleep(3)
    return 42
    
def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin, args=('thinking!', signal))
    print(f'spinner object: {spinner}')
    spinner.start()
    result = slow_function()
    signal.go = False
    spinner.join()
    return result
    
def main():
    result = supervisor()
    print(f'Answer: {result}')

if __name__ == '__main__':
    main()

Overwriting spinner_thread.py


А здесь - реализация того же функционала через конкурентность. Исходя из кода становится понятен принцип конкурентности, которым руководствуется `asyncio`. В коде программы функции заменяются корутинами (см. префикс `async` в определении функции), которые вызывают кроме прочего другие корутины (см. префикс `await` перед вызовом функции). Так мы даем `asyncio` понять, в каком месте можно приостановить выполнение функции и перейти к другой функции.  

In [15]:
%%writefile spinner_asyncio.py

import asyncio
import itertools
import sys

async def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush
    for char in itertools.cycle('|-/\\'):
        status = f'{char} {msg}'
        write(status)
        flush()
        write('\x08' * len(status))
        try:
            await asyncio.sleep(0.1)
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))

async def slow_function():
    await asyncio.sleep(3)
    return 42

async def supervisor():
    spinner = asyncio.create_task(spin('thinking!'))
    print(f'spinner object: {spinner}')
    result = await slow_function()
    spinner.cancel()
    return result

def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print(f'Answer = {result}')

if __name__ == '__main__':
    main()

Overwriting spinner_asyncio.py


В коде выше стоит обратить внимание на создание задачи (объект класса `Task`) методом `create_task()` библиотеки `asyncio`. Класс `Task` является дочерним по отношению к классу `asyncio.Future`, который является vis-a-vis класса `concurrent.futures.Future` (см. ноутбук `Futures.ipynb`) и реализует важный функционал. У данных классов есть принципиальное различие: у обоих есть метод `result()`. Если мы вызовем данный метод в незавершившем работу классе `asyncio.Future`, то в отличие от класса `concurrent.futures.Future` он не заблокирует поток в ожидании завершения работы футуры, а создаст исключение `asyncio.InvalidStateError`. Потому что ожидать выполнения футуры нужно через ключевое слово `await`.

А теперь попробуем с помощью `asyncio` повторить загрузчик флагов.

In [24]:
%%writefile flags_asyncio.py

import os
import time
import sys
import shutil
import asyncio
from httpx import AsyncClient

countries = 'CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR'.split()
base_url = 'http://flupy.org/data/flags'
dest_dir = './downloads'

def save_image(img, filename):
    path = os.path.join(dest_dir, filename)
    with open(path, 'wb') as fp:
        fp.write(img)

async def show(text):
    #Печатаем без перехода на следующую строку
    print(f'{text}', end=' ')

@asyncio.coroutine
def get_flag(country):
    url = f'{base_url}/{country}/{country}.gif'
    response = yield from aiohttp.request('GET', url)
    image = yield from response.read()
    return image

def init_dir(dest_dir):
    print('Initialization of download directory...')
    if os.path.exists(dest_dir):        
        shutil.rmtree(dest_dir)
    os.mkdir(dest_dir)
    print('Done.')
    
async def download_flag(country):
    image = await get_flag(country.lower())
    await save_image(image, f'{country.lower()}.gif')
    await show(country)
    return country

async def download_flags(country_lst):
    print('Downloading flags...')
    downloaded = []
    
    for country in sorted(country_lst):
        await download_flag(country)
        downloaded += [country]
    print('Done.')
    return len(downloaded)

def main(downloader):
    init_dir(dest_dir)
    start_time = time.time()
    count = asyncio.run(downloader(countries))
    time_taken = time.time() - start_time
    print(f'\n{count} flag(s) downloaded in {time_taken:.2f} seconds.')

if __name__ == '__main__':
    main(download_flags)

Overwriting flags_asyncio.py
